# Data Prepration

## Set Up

In [135]:
import os
import pandas as pd 
import numpy as np
import os.path as path
import glob 


## Data Preperation

In [136]:

def read_files():
    currentdir=os.getcwd()
    os.chdir("/home/shruthi/workenv/dataanalysis-shruthi")
    file_path=os.path.join(os.getcwd(), "data/raw/*.csv")
    print(file_path)
    all_files=glob.glob(file_path)
    print(len(all_files))
    final_df=pd.DataFrame()
    df=pd.DataFrame()
    filesnames_list=glob.glob(file_path)
    print(len(filesnames_list))
    #df=pd.concat((pd.read_csv(f, header=0) for f in filesnames_list), axis=0,ignore_index=True,join='inner',)
    final_df=pd.read_csv(all_files[0],header=0,parse_dates=False)
    for i in range(1,len(filesnames_list)):
        filename=all_files[i]
        df=pd.read_csv(filename,parse_dates=True)
    
        filname_split=filename.split('/')
        final_df= pd.concat([final_df,df],ignore_index=True,axis=0)
        final_df.drop(axis=0,labels=np.arange(13),inplace=True)
        final_df['user_id']=filname_split[-1].split(".")[0]
    return final_df

In [137]:
final_df=read_files()

/home/shruthi/workenv/dataanalysis-shruthi/data/raw/*.csv
22
22


In [138]:
final_df.shape

(31736, 22)

In [139]:
final_df.head()

,manufacturer,model,year,version,power(kW),fuel_date,odometer,trip_distance(km),quantity(kWh),fuel_type,tire_type,city,motor_way,country_roads,driving_style,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation,fuel_note,user_id
13,Volkswagen,Golf,2018,e-Golf,100,14.01.2022,NaN,59.0,6.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
14,Volkswagen,Golf,2018,e-Golf,100,13.01.2022,NaN,59.0,23.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
15,Volkswagen,Golf,2018,e-Golf,100,13.01.2022,NaN,49.0,7.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
16,Volkswagen,Golf,2018,e-Golf,100,13.01.2022,NaN,71.0,4.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
17,Volkswagen,Golf,2018,e-Golf,100,12.01.2022,NaN,59.0,20.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015


In [140]:
final_df.columns

Index(['manufacturer', 'model', 'year', 'version', 'power(kW)', 'fuel_date',
       'odometer', 'trip_distance(km)', 'quantity(kWh)', 'fuel_type',
       'tire_type', 'city', 'motor_way', 'country_roads', 'driving_style',
       'consumption(kWh/100km)', 'A/C', 'park_heating', 'avg_speed(km/h)',
       'ecr_deviation', 'fuel_note', 'user_id'],
      dtype='object')

In [141]:
data_sample=final_df[['power(kW)', 'fuel_date',
       'odometer', 'trip_distance(km)', 'quantity(kWh)', 'fuel_type',
       'tire_type', 'city', 'motor_way', 'country_roads', 'driving_style',
       'consumption(kWh/100km)', 'A/C', 'park_heating', 'avg_speed(km/h)',
       'ecr_deviation', 'fuel_note', 'user_id']]

In [142]:
data_sample.head()

,power(kW),fuel_date,odometer,trip_distance(km),quantity(kWh),fuel_type,tire_type,city,motor_way,country_roads,driving_style,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation,fuel_note,user_id
13,100,14.01.2022,NaN,59.0,6.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
14,100,13.01.2022,NaN,59.0,23.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
15,100,13.01.2022,NaN,49.0,7.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
16,100,13.01.2022,NaN,71.0,4.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
17,100,12.01.2022,NaN,59.0,20.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015


## Data Profiling

In [143]:
final_df[['manufacturer','model','year','version']].value_counts()

manufacturer   model     year  version                                  
Mitsubishi     i-MiEV    2013  i-MiEV                                       3211
Volkswagen     Golf      2014  e-Golf                                       2691
Opel           Ampera    2012  Ampera ePionier                              2563
Mitsubishi     i-MiEV    2011  i-MiEV                                       2185
Kia            Soul      2016  Kia Soul EV                                  2057
Opel           Ampera    2012  ePionier Lithiumweiß Elektrischer Antrieb    1727
Hyundai        IONIQ     2018  electric                                     1552
Volkswagen     Golf      2015  e-Golf                                       1392
                         2018  e-Golf                                       1372
Hyundai        IONIQ     2018  Roadrunner                                   1343
Mercedes-Benz  B-Klasse  2015  B250e                                        1310
Opel           Ampera    2017  Amper

In [144]:
final_df.head()

,manufacturer,model,year,version,power(kW),fuel_date,odometer,trip_distance(km),quantity(kWh),fuel_type,tire_type,city,motor_way,country_roads,driving_style,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation,fuel_note,user_id
13,Volkswagen,Golf,2018,e-Golf,100,14.01.2022,NaN,59.0,6.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
14,Volkswagen,Golf,2018,e-Golf,100,13.01.2022,NaN,59.0,23.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
15,Volkswagen,Golf,2018,e-Golf,100,13.01.2022,NaN,49.0,7.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
16,Volkswagen,Golf,2018,e-Golf,100,13.01.2022,NaN,71.0,4.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
17,Volkswagen,Golf,2018,e-Golf,100,12.01.2022,NaN,59.0,20.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015


In [145]:
final_df.shape

(31736, 22)

In [146]:
final_df.head(10)

,manufacturer,model,year,version,power(kW),fuel_date,odometer,trip_distance(km),quantity(kWh),fuel_type,tire_type,city,motor_way,country_roads,driving_style,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation,fuel_note,user_id
13,Volkswagen,Golf,2018,e-Golf,100,14.01.2022,NaN,59.0,6.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
14,Volkswagen,Golf,2018,e-Golf,100,13.01.2022,NaN,59.0,23.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
15,Volkswagen,Golf,2018,e-Golf,100,13.01.2022,NaN,49.0,7.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
16,Volkswagen,Golf,2018,e-Golf,100,13.01.2022,NaN,71.0,4.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
17,Volkswagen,Golf,2018,e-Golf,100,12.01.2022,NaN,59.0,20.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
18,Volkswagen,Golf,2018,e-Golf,100,12.01.2022,NaN,58.0,10.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
19,Volkswagen,Golf,2018,e-Golf,100,12.01.2022,NaN,59.0,5.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
20,Volkswagen,Golf,2018,e-Golf,100,11.01.2022,NaN,59.0,18.4,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
21,Volkswagen,Golf,2018,e-Golf,100,11.01.2022,NaN,58.0,7.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
22,Volkswagen,Golf,2018,e-Golf,100,11.01.2022,NaN,146.0,6.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015


In [147]:
final_df.columns

Index(['manufacturer', 'model', 'year', 'version', 'power(kW)', 'fuel_date',
       'odometer', 'trip_distance(km)', 'quantity(kWh)', 'fuel_type',
       'tire_type', 'city', 'motor_way', 'country_roads', 'driving_style',
       'consumption(kWh/100km)', 'A/C', 'park_heating', 'avg_speed(km/h)',
       'ecr_deviation', 'fuel_note', 'user_id'],
      dtype='object')

In [148]:
print_df=final_df[['manufacturer', 'model', 'year', 'version', 'trip_distance(km)', 'power(kW)','quantity(kWh)', 'fuel_type',
       'tire_type', 'city', 'motor_way', 'country_roads', 'driving_style',
       'consumption(kWh/100km)', 'A/C', 'park_heating', 'avg_speed(km/h)',
       'ecr_deviation']].copy()

In [149]:
pd.set_option('display.max_columns', None)

In [150]:
print_df=print_df.sample(frac=1)

In [151]:
print_df.head(10)

,manufacturer,model,year,version,trip_distance(km),power(kW),quantity(kWh),fuel_type,tire_type,city,motor_way,country_roads,driving_style,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation
17407,Opel,Ampera,2012,Ampera ePionier,99.0,111,0.75,Electricity,Summer tires,1,1,0,Normal,0.75,0,0,NaN,-12.75
19596,Opel,Ampera,2012,Ampera ePionier,48.0,111,13.3,Electricity,Winter tires,1,1,1,Normal,27.71,0,1,NaN,14.21
44,Volkswagen,Golf,2018,e-Golf,65.0,100,19.1,Electricity,All-year tires,0,1,0,Normal,20.29,1,0,NaN,7.389999999999999
20359,Mitsubishi,i-MiEV,2011,i-MiEV,104.5,49,12.4,Electricity,Summer tires,1,0,1,Normal,11.87,0,0,NaN,-1.6300000000000008
18820,Opel,Ampera,2012,Ampera ePionier,84.0,111,10.9,Electricity,Summer tires,1,1,0,Normal,12.98,1,0,NaN,-0.52
9292,Hyundai,IONIQ,2017,Premium,128.8,88,17.6,Electricity,Summer tires,1,1,1,Moderate,13.67,0,0,NaN,NaN
20004,Mitsubishi,i-MiEV,2011,i-MiEV,49.2,49,7.17,Electricity,Winter tires,1,0,1,Normal,13.77,0,0,NaN,0.2699999999999996
31244,Mercedes-Benz,B-Klasse,2015,B250e,65.0,132,8.9,Electricity,All-year tires,1,0,1,Normal,17.4,0,0,NaN,0.8
2385,Mitsubishi,i-MiEV,2014,i-MiEV,53.0,49,7.51,Green electricity,Winter tires,1,0,1,Normal,14.17,0,0,NaN,1.67
17281,Opel,Ampera,2012,Ampera ePionier,37.0,111,4.6,Electricity,Summer tires,1,0,1,Normal,12.43,0,0,NaN,-1.07


In [152]:
print(final_df['fuel_type'].value_counts())

fuel_type
Electricity          30376
Green electricity     1325
Super gasoline          31
fuel_type                2
E10                      2
Name: count, dtype: int64


In [153]:
print(final_df['city'].value_counts())

city
1       20812
0        8718
0        1437
1         767
city        2
Name: count, dtype: int64


In [154]:
print(final_df['motor_way'].value_counts())

motor_way
1            16240
0            13290
1             1618
0              586
motor_way        2
Name: count, dtype: int64


In [155]:
final_df['city'].unique()

array(['0', 'city', '1', 0, 1], dtype=object)

In [156]:
print(final_df['driving_style'].value_counts())

driving_style
Normal           20525
Moderate          9356
Fast              1742
driving_style        2
Name: count, dtype: int64


In [157]:
final_df.isna().sum()/final_df.shape[0]

manufacturer              0.000000
model                     0.000000
year                      0.000000
version                   0.000000
power(kW)                 0.000000
fuel_date                 0.000000
odometer                  0.260965
trip_distance(km)         0.000000
quantity(kWh)             0.000000
fuel_type                 0.000000
tire_type                 0.002836
city                      0.000000
motor_way                 0.000000
country_roads             0.000000
driving_style             0.003498
consumption(kWh/100km)    0.003151
A/C                       0.000000
park_heating              0.000000
avg_speed(km/h)           0.827861
ecr_deviation             0.131113
fuel_note                 0.621219
user_id                   0.000000
dtype: float64

In [158]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31736 entries, 13 to 31748
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   manufacturer            31736 non-null  object
 1   model                   31736 non-null  object
 2   year                    31736 non-null  object
 3   version                 31736 non-null  object
 4   power(kW)               31736 non-null  object
 5   fuel_date               31736 non-null  object
 6   odometer                23454 non-null  object
 7   trip_distance(km)       31736 non-null  object
 8   quantity(kWh)           31736 non-null  object
 9   fuel_type               31736 non-null  object
 10  tire_type               31646 non-null  object
 11  city                    31736 non-null  object
 12  motor_way               31736 non-null  object
 13  country_roads           31736 non-null  object
 14  driving_style           31625 non-null  object
 15  c

In [159]:
final_df[['trip_distance(km)', 'power(kW)','quantity(kWh)', 
       
       'consumption(kWh/100km)',  'avg_speed(km/h)',
       'ecr_deviation']].describe()

,trip_distance(km),power(kW),quantity(kWh),consumption(kWh/100km),avg_speed(km/h),ecr_deviation
count,31736.0,31736,31736.0,31636.00,5463.0,27575.00
unique,2192.0,14,4040.0,2791.00,117.0,3095.00
top,42.0,85,6.0,19.07,44.0,5.57
freq,1457.0,5372,246.0,222.00,232.0,215.00


In [160]:
final_df.columns

Index(['manufacturer', 'model', 'year', 'version', 'power(kW)', 'fuel_date',
       'odometer', 'trip_distance(km)', 'quantity(kWh)', 'fuel_type',
       'tire_type', 'city', 'motor_way', 'country_roads', 'driving_style',
       'consumption(kWh/100km)', 'A/C', 'park_heating', 'avg_speed(km/h)',
       'ecr_deviation', 'fuel_note', 'user_id'],
      dtype='object')

In [161]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31736 entries, 13 to 31748
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   manufacturer            31736 non-null  object
 1   model                   31736 non-null  object
 2   year                    31736 non-null  object
 3   version                 31736 non-null  object
 4   power(kW)               31736 non-null  object
 5   fuel_date               31736 non-null  object
 6   odometer                23454 non-null  object
 7   trip_distance(km)       31736 non-null  object
 8   quantity(kWh)           31736 non-null  object
 9   fuel_type               31736 non-null  object
 10  tire_type               31646 non-null  object
 11  city                    31736 non-null  object
 12  motor_way               31736 non-null  object
 13  country_roads           31736 non-null  object
 14  driving_style           31625 non-null  object
 15  c

### Converting Data types 

In [162]:
def convert_to_numeric(columns, df):
    for col in columns:
        df[col]=pd.to_numeric(df[col], errors='coerce')
        
colums_for_num_cov=['power(kW)','quantity(kWh)','consumption(kWh/100km)','trip_distance(km)','ecr_deviation','odometer']
convert_to_numeric(colums_for_num_cov, final_df) 
    

In [163]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31736 entries, 13 to 31748
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   manufacturer            31736 non-null  object 
 1   model                   31736 non-null  object 
 2   year                    31736 non-null  object 
 3   version                 31736 non-null  object 
 4   power(kW)               31734 non-null  float64
 5   fuel_date               31736 non-null  object 
 6   odometer                23452 non-null  float64
 7   trip_distance(km)       31734 non-null  float64
 8   quantity(kWh)           31734 non-null  float64
 9   fuel_type               31736 non-null  object 
 10  tire_type               31646 non-null  object 
 11  city                    31736 non-null  object 
 12  motor_way               31736 non-null  object 
 13  country_roads           31736 non-null  object 
 14  driving_style           31625 non-nul

In [164]:
final_df.head()

,manufacturer,model,year,version,power(kW),fuel_date,odometer,trip_distance(km),quantity(kWh),fuel_type,tire_type,city,motor_way,country_roads,driving_style,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation,fuel_note,user_id
13,Volkswagen,Golf,2018,e-Golf,100.0,14.01.2022,NaN,59.0,6.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
14,Volkswagen,Golf,2018,e-Golf,100.0,13.01.2022,NaN,59.0,23.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
15,Volkswagen,Golf,2018,e-Golf,100.0,13.01.2022,NaN,49.0,7.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
16,Volkswagen,Golf,2018,e-Golf,100.0,13.01.2022,NaN,71.0,4.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015
17,Volkswagen,Golf,2018,e-Golf,100.0,12.01.2022,NaN,59.0,20.0,Electricity,All-year tires,0,1,0,Normal,17.0,1,0,NaN,4.1,NaN,Mercedes_B250e_2015


In [165]:
final_df.columns


Index(['manufacturer', 'model', 'year', 'version', 'power(kW)', 'fuel_date',
       'odometer', 'trip_distance(km)', 'quantity(kWh)', 'fuel_type',
       'tire_type', 'city', 'motor_way', 'country_roads', 'driving_style',
       'consumption(kWh/100km)', 'A/C', 'park_heating', 'avg_speed(km/h)',
       'ecr_deviation', 'fuel_note', 'user_id'],
      dtype='object')

In [166]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31736 entries, 13 to 31748
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   manufacturer            31736 non-null  object 
 1   model                   31736 non-null  object 
 2   year                    31736 non-null  object 
 3   version                 31736 non-null  object 
 4   power(kW)               31734 non-null  float64
 5   fuel_date               31736 non-null  object 
 6   odometer                23452 non-null  float64
 7   trip_distance(km)       31734 non-null  float64
 8   quantity(kWh)           31734 non-null  float64
 9   fuel_type               31736 non-null  object 
 10  tire_type               31646 non-null  object 
 11  city                    31736 non-null  object 
 12  motor_way               31736 non-null  object 
 13  country_roads           31736 non-null  object 
 14  driving_style           31625 non-nul

In [167]:
# Shuffling data 
final_df=final_df.sample(frac=1)

In [168]:
final_df.shape

(31736, 22)

# Writing Consolidated dataframe to CSV

In [169]:
#final_df.to_parquet('prearedData.parquet', engine='fastparquet')
final_df.to_csv(os.getcwd()+'/data/prepared/prearedData.csv')